In [2]:
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F

# load data into pandas dfs
root_dir = '/kaggle/input/the-bards-best-a-character-modeling-dataset/'

train_file = 'train.csv'
test_file = 'test.csv'
val_file = 'validation.csv'

df_train = pd.read_csv(root_dir+train_file)
df_test = pd.read_csv(root_dir+test_file)
df_val = pd.read_csv(root_dir+val_file)


print(df_train)
print(df_test)
print(df_val)

                                                text
0  First Citizen:\nBefore we proceed any further,...
                                                text
0  rance ta'en\nAs shall with either part's agree...
                                                text
0  ?\n\nGREMIO:\nGood morrow, neighbour Baptista....


In [3]:
# parameters
batch_size = 32  # 64
block_size = 8  # 256
max_iter = 5000
eval_interval = 500
learning_rate = 1e-3  # 3e-4
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
eval_iters = 200
n_embed = 32  # 384
# n_head = 6
# n_layer = 6
# dropout = 0.2
head_size = 16

In [4]:
# explore datset and retrive characters in text
train_text = df_train['text'].iloc[0]
test_text = df_test['text'].iloc[0]
val_text = df_val['text'].iloc[0]

all_text = train_text+test_text+val_text
chars = sorted(list(set(all_text)))
vocab_size = len(chars)

# map chars to ints + reverse
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}

# enc/dec pair for int->str + str->int
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: [itos[i] for i in l]

# create encoded datasets
train_data = torch.tensor(encode(train_text), dtype=torch.long)
test_data = torch.tensor(encode(test_text), dtype=torch.long)
val_data = torch.tensor(encode(val_text), dtype=torch.long)

In [5]:
# get batch of tokens
def get_batch(split):
    data = train_data if split == 'train' else val_data
    idx = torch.randint(len(data)-block_size, (batch_size, ))
    
    x = torch.stack([data[i:i+block_size] for i in idx])
    y = torch.stack([data[i+1:i+block_size+1] for i in idx])
    return x.to(device), y.to(device)

@torch.no_grad()
def estim_loss():
    out = {}
    model.eval()
    
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        
        for k in range(eval_iters):
            X, y = get_batch(split)
            logits, loss = model(X, y)
            losses[k] = loss.item()
            
        out[split] = losses.mean()
    model.train()
    return out

xb, yb = get_batch('train')


In [6]:
class SingleHeadAttention(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        
    def forward(self, x):
        B, T, C = x.shape

        k = self.key(x)
        q = self.query(x)
        v = self.value(x)

        w = q@k.transpose(-2, -1)
        w = w.masked_fill(self.tril[:T, :T]==0, float('-inf'))
        w = F.softmax(w, dim=-1)*head_size**-0.5

        out = w@v
        return out

    
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([SingleHeadAttention(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embed, n_embed)
    
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        return out

    
class FeedForward(nn.Module):
    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4*n_embed),
            nn.ReLU(),
            nn.Linear(4*n_embed, n_embed)
        )
    
    def forward(self, x):
        return self.net(x)

    
class TransformerBlock(nn.Module):
    def __init__(self, n_embed, num_heads):
        super().__init__()
        head_size = n_embed//num_heads
        self.sa = MultiHeadAttention(num_heads, head_size)
        self.ffwd = FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)
        
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


In [7]:
class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        
        self.transformer_blocks = nn.Sequential(
            TransformerBlock(n_embed, num_heads=4),
            TransformerBlock(n_embed, num_heads=4),
            TransformerBlock(n_embed, num_heads=4),
            nn.LayerNorm(n_embed),
        )
        self.ffwd = FeedForward(n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)
    
    def forward(self, idx, targets=None):
        B, T = idx.shape
        tokens_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        
        x = tokens_emb + pos_emb
        x = self.transformer_blocks(x)
        logits = self.lm_head(x)
    
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            print(idx_cond)
            logits, loss = self(idx_cond)
            logits = logits[: -1, :]
            probs = F.softmax(logits, dim=-1)
            print(probs)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, idx_next], dim=1)
        return idx

In [8]:
model = BigramLanguageModel().to(device)

optimiser = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for i in range(max_iter):
    if (i+1) % eval_interval == 0:
        losses = estim_loss()
        print(f'step {i+1:4d}: train loss {losses["train"]:.4f}, val loss {losses["val"]:.4f}')
        
    xb, yb = get_batch('train')
    
    logits, loss = model(xb, yb)
    optimiser.zero_grad(set_to_none=True)
    loss.backward()
    optimiser.step()

step  500: train loss 2.3963, val loss 2.3826
step 1000: train loss 2.2648, val loss 2.2718
step 1500: train loss 2.2065, val loss 2.2187
step 2000: train loss 2.1526, val loss 2.1648
step 2500: train loss 2.1105, val loss 2.1382
step 3000: train loss 2.0809, val loss 2.1156
step 3500: train loss 2.0466, val loss 2.1006
step 4000: train loss 2.0384, val loss 2.1010
step 4500: train loss 2.0149, val loss 2.0656
step 5000: train loss 2.0047, val loss 2.0675
